In [1]:
import numpy as np
import time
import copy

# Auxiliar Functions

## get_neighbors()

In [2]:
def get_neighbors(index, clockwise):
    neighbors = []
    if clockwise:
        neighbors.append((index - 1) % 8)
        neighbors.append((index + 1) % 8)
    else:
        neighbors.append((index + 1) % 8)
        neighbors.append((index - 1) % 8)
    return neighbors

## is_corner()

In [3]:
def is_corner(dir):
    switch = {
        0: True,
        1: False,
        2: True,
        3: False,
        4: True,
        5: False,
        6: True,
        7: False
    }
    return switch.get(dir, False)

In [4]:
if (is_corner(0)):
    print("Corner")
else:
    print("Not a corner")

Corner


## valid_position()

In [5]:
def valid_position(row, col, shape):
    height, width, aux = shape
    if row == -1:
        row = height - 1
    elif row == height:
        row = 0

    if col == -1:
        col = width - 1
    elif col == width:
        col = 0
    
    return row, col

## get_animals_around()

In [6]:
# [row, column, [rabbits, foxes]]

# 0 1 2
# 7   3
# 6 5 4

def get_animals_around(grid, row, col, animal):

    if animal == "rabbit":
        animal_index = 0
    elif animal == "fox":
        animal_index = 1

    height, width = grid.shape[:2]
    if valid_position(row, col, grid.shape) != (row, col):
        print("Invalid position: ({}, {})".format(row, col))
        return None

    map = np.zeros(8, dtype=int)
    
    for i in range(3):
        valid_row, valid_col = valid_position(row - 1, col - 1 + i, grid.shape)
        map[i] = grid[valid_row, valid_col, animal_index]
      
    valid_row, valid_col = valid_position(row, col + 1, grid.shape)
    map[3] = grid[valid_row, valid_col, animal_index]    

    for i in range(3):
        valid_row, valid_col = valid_position(row + 1, col + 1 - i, grid.shape)
        map[4 + i] = grid[valid_row, valid_col, animal_index]    
    
    valid_row, valid_col = valid_position(row, col - 1, grid.shape)
    map[7] = grid[valid_row, valid_col, animal_index]

    return map

## Grids

In [7]:
# [row, column, [rabbits, foxes]]
height = 3
width = 4

grid = np.zeros((height, width, 2), dtype=int)

c = 0
for row in range(height):
    for col in range(width):
        grid[row, col, 0] = c
        grid[row, col, 1] = c + 1
        c += 2
grid

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]],

       [[16, 17],
        [18, 19],
        [20, 21],
        [22, 23]]])

In [8]:
# height = 3
# width = 4

# 0 1 2
# 7   3
# 6 5 4

# 0  2  4  6
# 8  10 12 14
# 16 18 20 22


get_animals_around(grid, 2, 3, "rabbit")

array([12, 14,  8, 16,  0,  6,  4, 20])

In [9]:
# [row, column, [rabbits, foxes]]
height = 3
width = 4

# Create a map. Each cell is a tuple (rabbits, foxes)
# rabbits are represented by even numbers
# foxes are represented by odd numbers
grid = np.zeros((height, width, 2), dtype=int)

c = 0
for row in range(height):
    for col in range(width):
        grid[row, col, 0] = c
        grid[row, col, 1] = c + 1
        c += 2
grid

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]],

       [[16, 17],
        [18, 19],
        [20, 21],
        [22, 23]]])

In [10]:
# [row, column, [rabbits, foxes]]
width = 2
height = 3

grid = np.zeros((height, width, 2), dtype=int)

grid[0,0,0] = 1
grid[0,0,1] = 2

grid[0,1,0] = 3
grid[0,1,1] = 0

grid[1,0,0] = 2
grid[1,0,1] = 1

grid[1,1,0] = 0
grid[1,1,1] = 3

grid[2,0,0] = 3
grid[2,0,1] = 3

grid[2,1,0] = 3
grid[2,1,1] = 5

grid 

array([[[1, 2],
        [3, 0]],

       [[2, 1],
        [0, 3]],

       [[3, 3],
        [3, 5]]])

In [11]:
np.array(
    [[[1,2], [3,0]],
    [[2,1], [0,3]],
    [[3,3], [3,5]]] )

array([[[1, 2],
        [3, 0]],

       [[2, 1],
        [0, 3]],

       [[3, 3],
        [3, 5]]])

# Classes

## class Strategy

In [12]:
class Strategy:

    # This method receives an array
    # of length 8, the amount of foxes around 
    # and returns the next state.
    # The current state (f) is an array of eight integers, 
    # f[i] := amount of foxes in the i-th direction
    # state index, same as directions:
    # 0 1 2
    # 7   3
    # 6 5 4
    @staticmethod
    def clockwise_escape(f, preferred_direction, clockwise):
        neighbors = get_neighbors(preferred_direction, clockwise)
        if not ( f[preferred_direction] 
                or f[neighbors[0]] 
                or f[neighbors[1]] ):
            return preferred_direction # safe from visible foxes

        current_direction = preferred_direction
        evaluated_safer_directions = 1
        while evaluated_safer_directions < 8:
            if clockwise:
                current_direction = (current_direction + 1) % 8
            else:
                current_direction = (current_direction - 1) % 8

            neighbors = get_neighbors(current_direction, clockwise)
            if not ( f[current_direction] 
                    or f[neighbors[0]] 
                    or f[neighbors[1]] ):
                return current_direction # safe from visible foxes
            evaluated_safer_directions += 1

        if not (f[preferred_direction]):
            return preferred_direction

        evaluated_unsafe_directions = 1
        while evaluated_unsafe_directions < 8:
            if clockwise:
                current_direction = (current_direction + 1) % 8
            else:
                current_direction = (current_direction - 1) % 8

            if not (f[current_direction]):
                return current_direction
            evaluated_unsafe_directions += 1
        
        return np.argmin(f)


    # This function receives the current state (r) of 
    # the system and returns the next state.
    # The current state (r) is an array of eight integers, 
    # r[i] := amount of rabbits in the i-th direction
    # state index, same as directions:
    # 0 1 2
    # 7   3
    # 6 5 4

    @staticmethod
    def chasing_1(r, preferred_direction, clockwise, diagonal_prediction):
        if not( np.any(r) ):
            return preferred_direction
        best_direction = np.argmax(r)
        neighbors = get_neighbors(best_direction, clockwise)
        if is_corner( best_direction ) or not( diagonal_prediction ):
            return neighbors[1]
        else:
            return get_neighbors(neighbors[1], clockwise)[1]

In [13]:
Strategy.clockwise_escape([10, 0, 1, 1, 3, 1, 0, 2], 5, False)

1

In [14]:
# 0 1 2
# 7   3
# 6 5 4
r = [0, 0, 2, 0, 0, 0, 0, 1]

Strategy.chasing_1(r, 0, True, False)

3

In [15]:
rabbit_amounts = [0, 0, 1, 3, 4, 1, 0, 0]
Strategy.chasing_1(rabbit_amounts, 2, clockwise = True, diagonal_prediction = True)

5

## class Animal

In [16]:
class Animal:

    def __init__(self, _row, _col, _species, _strategy, 
        _preferred_direction = None, _clockwise = True,
        _diagonal_prediction = False):

        if _species == "rabbit" or _species == "fox":
            self.species = _species
        else:
            raise ValueError("invalid animal species")
        
        self.row = _row
        self.col = _col
        self.strategy = _strategy
        self.preferred_direction = _preferred_direction
        self.clockwise = _clockwise
        self.diagonal_prediction = _diagonal_prediction

        if _species == "rabbit":
            self.animal_index = 0
        elif _species == "fox":
            self.animal_index = 1


    def get_position(self):
        return (self.row, self.col)
    

    def get_strategy(self):
        return self.strategy


    def move(self, shape, rabbits_around = None, foxes_around = None):
        if self.strategy == "random":
            dir =  np.random.randint(0, 8)


        elif self.strategy == "clockwise_escape":
            dir = Strategy.clockwise_escape(foxes_around,
                self.preferred_direction, self.clockwise)


        elif self.strategy == "chasing_1":
            dir = Strategy.chasing_1(rabbits_around, self.preferred_direction,
                self.clockwise, self.diagonal_prediction)
        else:
            raise ValueError("invalid strategy")

        # 0 1 2
        # 7   3
        # 6 5 4

        if (dir < 0) or (dir > 7):
            raise ValueError("Animal.move(): invalid direction")

        if dir == 0:
            self.row -= 1
            self.col -= 1
        elif dir == 1:
            self.row -= 1
        elif dir == 2:
            self.row -= 1
            self.col += 1
        elif dir == 3:
            self.col += 1
        elif dir == 4:
            self.row += 1
            self.col += 1
        elif dir == 5:
            self.row += 1
        elif dir == 6:
            self.row += 1
            self.col -= 1
        elif dir == 7:
            self.col -= 1 

        self.row, self.col = valid_position(self.row, self.col, shape)
    

    def reproduce(self):
        return Animal(self.row, self.col, self.species, self.strategy,
            self.preferred_direction, self.clockwise, self.diagonal_prediction)


In [17]:

rabbit_1 = Animal(1, 3, "rabbit", "random")

In [18]:
rabbit_1.species

'rabbit'

In [19]:
rabbit_1.get_position()


(1, 3)

In [20]:
rabbit_2 = rabbit_1.reproduce()
rabbit_2.species

'rabbit'

## class Map

In [21]:
class Map:

    def __init__(self, _rows, _cols):
        self.species_list = ["rabbit", "fox"]
        self.rows = _rows
        self.cols = _cols
        self.population_map = np.zeros(
            (self.rows, self.cols, len(self.species_list) ), dtype = int)

        self.objects_map = [[[[], []] for i in range(self.cols)] for j in range(self.rows)]
        self.hunt_log_map = np.zeros((self.rows, self.cols), dtype = int)


    def get_population_map(self):
        return self.make_population_map()


    def get_objects_map(self):
        return self.objects_map


    def set_population_map(self, _population_map):
        self.population_map = _population_map


    def set_objects_map(self, _objects_map):
        self.objects_map = _objects_map


    def populate(self, amount, species, strategy, 
        preferred_direction = None, clockwise = True,
        diagonal_prediction = False):

        self.hunt_log_map = np.zeros((self.rows, self.cols), dtype = int)

        if not (species == "rabbit" or species == "fox"):
            raise ValueError("invalid animal species")

        if species == "rabbit":
            species_index = 0
        elif species == "fox":
            species_index = 1

        for i in range (amount):
            row = np.random.randint(0, self.rows)
            col = np.random.randint(0, self.cols)
            self.objects_map[row][col][species_index].append(Animal(
                row, col, species, strategy, preferred_direction, 
                clockwise, diagonal_prediction))
        self.make_population_map()

    def move_animals(self):
        self.make_population_map()
        
        for i in range(self.rows):
            for j in range(self.cols):
                for species_index in range( len(self.species_list) ):
                    for animal in self.objects_map[i][j][species_index]:
                        rabbits_around = None
                        foxes_around = None
                        if animal.species == "rabbit":
                            if species_index != 0:
                                raise ValueError("invalid animal index")
                            foxes_around = get_animals_around(self.population_map, 
                                i, j, "fox" )
                        elif (animal.species == "fox"):
                            if species_index != 1:
                                raise ValueError("invalid animal index")
                            rabbits_around = get_animals_around(self.population_map, 
                                i, j, "rabbit" )
                        animal.move( (self.rows, self.cols, len(self.species_list)), 
                            rabbits_around, foxes_around )
                        self.objects_map[animal.row][animal.col][species_index].append(animal)
                        self.objects_map[i][j][species_index].remove(animal)


    def make_population_map(self):
        self.population_map = np.zeros(
            (self.rows, self.cols, len(self.species_list) ), dtype = int)

        for i in range(self.rows):
            for j in range(self.cols):
                for species_index in range( len(self.species_list) ):
                    self.population_map[i, j, species_index] \
                        = len(self.objects_map[i][j][species_index])
        return self.population_map


    def hunt(self):
        self.hunt_log_map = np.zeros((self.rows, self.cols), dtype = int)
        for i in range(self.rows):
            for j in range(self.cols):
                rabbits_array = self.objects_map[i][j][0]
                foxes_array = self.objects_map[i][j][1]

                hunted_animals = min( len(foxes_array), len(rabbits_array) )
                self.objects_map[i][j][0] = list (np.delete(
                        rabbits_array, np.random.choice(len(rabbits_array), 
                    hunted_animals, replace = False)))
                self.hunt_log_map[i, j] += hunted_animals


    def reproduction(self, species):
        temporal_objects_map = copy.deepcopy(self.objects_map)

        if species == "rabbit":
            for i in range(self.rows):
                for j in range(self.cols):
                    species_index = self.species_list.index(species)
                    for animal in temporal_objects_map[i][j][species_index]:
                        brood = animal.reproduce()
                        self.objects_map[brood.row][brood.col][species_index] \
                            .append(animal.reproduce())

        elif species == "fox":
            for i in range(self.rows):
                for j in range(self.cols):
                    species_index = self.species_list.index(species)
                    print(f"self.objects_map[i][j][species_index] \
                        = {self.objects_map[i][j][species_index]}")
                    print(f"self.hunt_log_map[i, j] = {self.hunt_log_map[i, j]}")
                    print(f"species_index = {species_index}")
                    if (self.objects_map[i][j][species_index]):
                        breeding_foxes = list( np.random.choice(
                            self.objects_map[i][j][species_index],
                            self.hunt_log_map[i, j],
                            replace = False ) )
                        for fox in breeding_foxes:
                            brood = fox.reproduce()
                            self.objects_map[brood.row][brood.col][species_index] \
                                .append(fox.reproduce())


In [22]:

m1 = Map(3, 3)
m1.populate(10, "rabbit", "random")
m1.get_population_map()


array([[[1, 0],
        [4, 0],
        [0, 0]],

       [[0, 0],
        [3, 0],
        [0, 0]],

       [[1, 0],
        [0, 0],
        [1, 0]]])

In [23]:

m2 = Map(3, 3)
m2.populate(10, "rabbit", "random")
m2.populate(10, "fox", "random")
m2.get_population_map()


array([[[0, 0],
        [0, 4],
        [2, 0]],

       [[2, 0],
        [1, 0],
        [1, 1]],

       [[1, 0],
        [2, 3],
        [1, 2]]])

In [24]:
m2.hunt()
print(m2.hunt_log_map)
print(m2.get_population_map())

[[0 0 0]
 [0 0 1]
 [0 2 1]]
[[[0 0]
  [0 4]
  [2 0]]

 [[2 0]
  [1 0]
  [0 1]]

 [[1 0]
  [0 3]
  [0 2]]]


In [57]:
m2.move_animals()
m2.get_population_map()

array([[[2, 1],
        [0, 1],
        [3, 2]],

       [[2, 1],
        [0, 0],
        [2, 3]],

       [[1, 0],
        [1, 1],
        [1, 1]]])

In [26]:
# m2.hunt()
# m2.reproduction("fox")
m2.get_population_map()

array([[[0, 1],
        [2, 3],
        [1, 1]],

       [[2, 1],
        [0, 1],
        [0, 0]],

       [[0, 0],
        [1, 2],
        [0, 1]]])

In [27]:
np.random.choice([3, 5], 0, replace = False)

array([], dtype=int64)

In [28]:
m2.reproduction("rabbit")
m2.get_population_map()

array([[[0, 1],
        [4, 3],
        [2, 1]],

       [[4, 1],
        [0, 1],
        [0, 0]],

       [[0, 0],
        [2, 2],
        [0, 1]]])

In [29]:
l_o_l = [[[1, 5], [2, 4]], [3, 4], [5, 6]]
a = l_o_l[0][0]
a

[1, 5]

In [30]:
a.remove(1)

In [31]:
a = [5]

In [32]:
a

[5]

In [33]:
l_o_l

[[[5], [2, 4]], [3, 4], [5, 6]]

In [34]:
rabbits_array = [1, 2, 3, 4, 5]

In [35]:
rabbits_array = list (np.delete(
                        rabbits_array, np.random.choice(len(rabbits_array), 
                    3, replace = False)))

In [36]:
# rabbits_array.pop(list(np.random.choice(len(rabbits_array), 3, replace = False)))

In [37]:
np.random.choice(3, 2, replace = False)

array([0, 2])

In [38]:
rabbits_array

[2, 3]

# temp

In [39]:
class MyClass:
    classVar = None

    def __init__(self, x, newClassVar = None):
        self.x = x
        if newClassVar is not None:
            MyClass.classVar = newClassVar
    
    def setClassVar(self, newClassVar):
        MyClass.classVar = newClassVar
        print(f"getClassVar: {self.getClassVar()}")

    def getClassVar(self):
        return MyClass.classVar

    def print_message_then_wait(self, message):
        print(message)
        time.sleep(5)
        print(f"x: {self.x}")

In [40]:
rows = 2
cols = 3

my_map = [[[[], []] for i in range(cols)] for j in range(rows)]
my_map

[[[[], []], [[], []], [[], []]], [[[], []], [[], []], [[], []]]]

In [41]:
animal_index = 0
my_map[1][0][ animal_index ].append(a)
my_map

[[[[], []], [[], []], [[], []]], [[[[5]], []], [[], []], [[], []]]]

In [42]:
my_map[1][0][ animal_index ].remove(a)
my_map

[[[[], []], [[], []], [[], []]], [[[], []], [[], []], [[], []]]]

In [43]:
arr = list(np.arange(10))
print( type(arr) )
# append 10 to arr
arr = list(np.append(arr, 10))
arr
print( type(arr) )

<class 'list'>
<class 'list'>


In [44]:
arr = list(np.append(arr, 15))
type( list(arr) )
print(arr.index(15))
arr


11


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15]

In [45]:
# np.random.choice(len(arange), 2, replace = False)

In [46]:
arange = np.arange(10)

In [47]:
arange = np.delete(arange, np.random.choice(len(arange), 2 , replace = False))

arange 

array([0, 1, 2, 3, 6, 7, 8, 9])